## Setup Environment

In [ ]:
!pip install ultralytics
!pip install opencv-python

In [ ]:
from ultralytics import YOLO , solutions
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the YOLO Model

In [ ]:
model = YOLO('/content/YOLO_Model_v5.pt')

## Prepare the Video Capture

In [ ]:
# Prepare the video capture
video_path = '/content/drive/MyDrive/Dataset/IMG_9462.MOV'
cap = cv2.VideoCapture(video_path)

## Get Video Information

In [ ]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [ ]:
print("Height:", height)
print("Width:", width)
print("FPS:", fps)

Height: 1584
Width: 2816
FPS: 59


In [ ]:
output_path = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [ ]:
License_plate = set()
vehicle = set()

In [ ]:
box = (1650, 900, 2816, 1500)
box2 = (300, 300, 2816, 1500)


In [ ]:
counter = 0
counter_vehicle = 0

In [ ]:
print(model.names)

{0: 'License_Plate', 1: 'cars', 2: 'motorcycle', 3: 'truck'}


In [ ]:
# # Define the region near the bottom of the frame
# close_to_camera_region = (int(height * 0.8), height)  # Adjust the region as needed
class_names = ['License_Plate', 'Car', 'motorcycle', 'truck']
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model on the frame
    results = model.track(frame, persist=True)

    for result in results:
        for boxes in result.boxes:

            # Get Object details
            bbox = boxes.xyxy[0].cpu().numpy()          # Bounding Box
            class_id = int(boxes.cls[0].cpu().numpy())  # Class ID
            conf = boxes.conf[0].cpu().numpy()          # Confidence

            # Check if boxes.id is not None before accessing it
            if boxes.id is not None:
                id = int(boxes.id[0].cpu().numpy())     # Object ID
            else:
                # Handle the case where id is None
                id = -1

            x1, y1, x2, y2 = map(int, bbox)
            cropped_object = frame[y1:y2, x1:x2]

            # Draw BBox
            cv2.rectangle(frame, (x1, y1), (x2, y2), (208, 38, 7), 3)

            label = f'ID: {id}, class: {class_names[class_id]} Conf: {conf:.2f}'
            # Add labels for each car
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (208, 38, 7), 2, cv2.LINE_AA)

            # # Detect car and add the id to the set
            # if id not in License_plate:
            #     License_plate.add(id)
            #     save_path = os.path.join('/content/vid 3 images', f"{counter}.jpg")  # Save with object name and index
            #     counter += 1
            #     # Save the cropped image
            #     cv2.imwrite(save_path, cropped_object)

            # Check if the object is close to the camera
            # if y2 >= close_to_camera_region[0]:
            #     if id not in License_plate:
            #       License_plate.add(id)
            #       save_path = os.path.join('/content/Images1', f"{counter}.jpg")  # Save with object name and index
            #       counter += 1
            #        # Save the cropped image
            #       cv2.imwrite(save_path, cropped_object)
            if x1 >= box[0] and y1 >= box[1] and x2 <= box[2] and y2 <= box[3]:
                if id not in License_plate:
                    License_plate.add(id)
                    save_path = os.path.join('/content/Images', f"{counter}.jpg")  # Save with object name and index
                    counter += 1
                    # Save the cropped image
                    cv2.imwrite(save_path, cropped_object)
            if x1 >= box2[0] and y1 >= box2[1] and x2 <= box2[2] and y2 <= box2[3]:
                if id not in vehicle:
                    vehicle.add(id)
                    save_path_vehicle = os.path.join('/content/Images2', f"{counter_vehicle}.jpg")  # Save with object name and index
                    counter_vehicle += 1
                    # Save the cropped image
                    cv2.imwrite(save_path_vehicle, cropped_object)


    # cv2.rectangle(frame, box[:2], box[2:], (0, 0, 255), 2)


    video_writer.write(frame)

    # Display each frame
    # cv2_imshow(frame)


Streaming output truncated to the last 5000 lines.
0: 384x640 3 carss, 374.1ms
Speed: 8.2ms preprocess, 374.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 carss, 370.0ms
Speed: 4.8ms preprocess, 370.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 carss, 386.9ms
Speed: 4.7ms preprocess, 386.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 carss, 381.9ms
Speed: 5.1ms preprocess, 381.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 carss, 585.7ms
Speed: 10.3ms preprocess, 585.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 carss, 593.4ms
Speed: 5.5ms preprocess, 593.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 carss, 579.8ms
Speed: 4.6ms preprocess, 579.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 carss, 614.6ms
Speed: 6.1ms preprocess, 614.6m

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()
